# HISTORISATION DES LEADS

### Imports & data extraction

In [123]:
# imports des librairies python qu'on utilise
import pandas as pd
import numpy as np

In [124]:
# on retrieve la data de l'enorme export VW contact de plusieurs années
df = pd.read_excel('../data/bulk_data_leads.xlsx')
df.head()

,ID Contact,Campagne,Source,Date,Statut,Tracking
0,588dd2f7-cee7-44b8-b840-a373b5a1e94e,VP_RDV_SOCIAL_FACEBOOK,882307428996935,2020-12,-1.0,No callTracking
1,a09ee853-c20a-421a-bc24-d463ea972baa,VP_RDV_SOCIAL_FACEBOOK,Sans source,2019-12,-1.0,Non rappelé
2,5b7a90dd-1acc-4ae7-b12d-03cdfd76b290,VP_RDV_SOCIAL_FACEBOOK,Sans source,2021-5,3.0,No callTracking
3,d258ab29-b427-450b-b655-5ec9ac0f2e21,VP_RDV_SOCIAL_FACEBOOK,Sans source,2020-2,3.0,No callTracking
4,b853cc38-afac-483f-b0a8-c2eaea4570f7,VP_RDV_SOCIAL_FACEBOOK,Sans source,2020-2,-1.0,No callTracking


In [125]:
# shape de la data, combien de lignes et de colonnes
df.shape

(708339, 6)

### Process d'automatisation sur un mois

In [156]:
# focus sur le mois de aout 2019, on subset ce mois à la raw data
raw_month = df[df['Date'].str.contains('2019-8')]
raw_month.head(3)

,ID Contact,Campagne,Source,Date,Statut,Tracking
16,9d14672a-0f36-4209-8132-f85af036f38e,VP_RDV_SOCIAL_FACEBOOK,Sans source,2019-8,3.0,No callTracking
41,c1f47ac5-5321-4375-8e64-cfe440b7f6ea,VP_RDV_AFFILIATION_MAVOITUREPARINTERNET,Sans source,2019-8,NaN,No callTracking
93,209f2496-5241-4132-a28e-0245a579e238,VP_RDV_SOCIAL_FACEBOOK,Sans source,2019-8,-1.0,No callTracking


In [159]:
# creation et mise en forme du dataframe mensuel qu'on va utiliser pour chaque mois de focus
base = {'Month': [], 'Focus': [], 'Direct': [], 'LSTD': [], 'Total': []}
df_template = pd.DataFrame(data = base)
df_template.at[0, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'Affiliation'
df_template.at[1, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'Lead_Ad'
df_template.at[2, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'Operations'
df_template.at[4, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'VW_SEA'
df_template.at[5, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'VW_Display'
df_template.at[6, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'VW_Paid'
df_template.at[7, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'VW_Organic'
df_template.at[8, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'Dealers'
df_template.at[9, 'Month'] = 'idk'
df_template.loc[df_template['Focus'].isnull(), 'Focus'] = 'TOTAL MONTH'

df_template

,Month,Focus,Direct,LSTD,Total
0,idk,Affiliation,NaN,NaN,NaN
1,idk,Lead_Ad,NaN,NaN,NaN
2,idk,Operations,NaN,NaN,NaN
4,idk,VW_SEA,NaN,NaN,NaN
5,idk,VW_Display,NaN,NaN,NaN
6,idk,VW_Paid,NaN,NaN,NaN
7,idk,VW_Organic,NaN,NaN,NaN
8,idk,Dealers,NaN,NaN,NaN
9,idk,TOTAL MONTH,NaN,NaN,NaN


In [193]:
# là c'est mon bidouillage pour trouver une méthode qui marche, à ignorer
df_month = df_template.copy()
current_date = '2019-8'

var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_AFFILIATION_MEETDEAL')) & (raw_month['Statut'] == 1)].shape[0]
df_month.loc[df_month['Focus'] == 'Affiliation', 'Direct'] = var1
var2 = raw_month[(raw_month['Campagne'] != 'VP_RDV_AFFILIATION_MEETDEAL') & (raw_month['Campagne'].str.contains('VP_RDV_AFFILIATION')) & (raw_month['Statut'] == 1)].shape[0]
df_month.loc[df_month['Focus'] == 'Affiliation', 'LSTD'] = var2

var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_SOCIAL')) & (raw_month['Statut'] == 1)].shape[0]
df_month.loc[df_month['Focus'] == 'Lead_Ad', 'Direct'] = 0
df_month.loc[df_month['Focus'] == 'Lead_Ad', 'LSTD'] = var1

var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_OPERATIONS')) & (raw_month['Statut'] == 1)].shape[0]
df_month.loc[df_month['Focus'] == 'Operations', 'Direct'] = var1
df_month.loc[df_month['Focus'] == 'Operations', 'LSTD'] = 0

var1 = raw_month[(~raw_month['Campagne'].str.contains('MEETDEAL')) & (raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('SEA'))].shape[0]
df_month.loc[df_month['Focus'] == 'VW_SEA', 'Direct'] = var1
df_month.loc[df_month['Focus'] == 'VW_SEA', 'LSTD'] = 0

var1 = raw_month[(~raw_month['Campagne'].str.contains('MEETDEAL')) & (raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('DIS|DSP|NAT'))].shape[0]
df_month.loc[df_month['Focus'] == 'VW_Display', 'Direct'] = 0
df_month.loc[df_month['Focus'] == 'VW_Display', 'LSTD'] = var1

var1 = raw_month[(~raw_month['Campagne'].str.contains('MEETDEAL')) & (raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('SOC'))].shape[0]
df_month.loc[df_month['Focus'] == 'VW_Paid', 'Direct'] = 0
df_month.loc[df_month['Focus'] == 'VW_Paid', 'LSTD'] = var1

var1 = raw_month[(raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('Sans source'))].shape[0]
df_month.loc[df_month['Focus'] == 'VW_Organic', 'Direct'] = var1
df_month.loc[df_month['Focus'] == 'VW_Organic', 'LSTD'] = 0

var1 = raw_month[raw_month['Campagne'].str.contains('VP_RDV_DISTRIBUTEURS_SITES')].shape[0]
df_month.loc[df_month['Focus'] == 'Dealers', 'Direct'] = var1
df_month.loc[df_month['Focus'] == 'Dealers', 'LSTD'] = 0

df_month.loc[df_month['Focus'] == 'TOTAL MONTH', 'Direct'] = df_month.head(8)['Direct'].sum()
df_month.loc[df_month['Focus'] == 'TOTAL MONTH', 'LSTD'] = df_month.head(8)['LSTD'].sum()
df_month['Total'] = df_month['Direct'] + df_month['LSTD']

df_month[['Direct', 'LSTD', 'Total']] = df_month[['Direct', 'LSTD', 'Total']].astype(int).round()
df_month['Month'] = current_date

df_month

,Month,Focus,Direct,LSTD,Total
0,2019-8,Affiliation,0,5601,5601
1,2019-8,Lead_Ad,0,508,508
2,2019-8,Operations,1020,0,1020
4,2019-8,VW_SEA,3021,0,3021
5,2019-8,VW_Display,0,204,204
6,2019-8,VW_Paid,0,0,0
7,2019-8,VW_Organic,8005,0,8005
8,2019-8,Dealers,254,0,254
9,2019-8,TOTAL MONTH,12300,6313,18613


## Maintenant le reste des mois

In [194]:
# definition d'une fonction, qui en fonction du mois qu'on lui donne, nous renvoie le tableau correspondant
def histo_leads(current_date):
    raw_month = df[df['Date'].str.contains(current_date)]
    raw_month.head(3)
    df_month = df_template.copy()
    var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_AFFILIATION_MEETDEAL')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'Affiliation', 'Direct'] = var1
    var2 = raw_month[(raw_month['Campagne'] != 'VP_RDV_AFFILIATION_MEETDEAL') & (raw_month['Campagne'].str.contains('AFFILIATION')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'Affiliation', 'LSTD'] = var2
    var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_SOCIAL')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'Lead_Ad', 'Direct'] = 0
    df_month.loc[df_month['Focus'] == 'Lead_Ad', 'LSTD'] = var1
    var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_OPERATIONS')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'Operations', 'Direct'] = var1
    df_month.loc[df_month['Focus'] == 'Operations', 'LSTD'] = 0
    var1 = raw_month[(~raw_month['Campagne'].str.contains('MEETDEAL')) & (raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('SEA')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'VW_SEA', 'Direct'] = var1
    df_month.loc[df_month['Focus'] == 'VW_SEA', 'LSTD'] = 0
    var1 = raw_month[(~raw_month['Campagne'].str.contains('MEETDEAL')) & (raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('_DIS|_DSP|_NAT')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'VW_Display', 'Direct'] = 0
    df_month.loc[df_month['Focus'] == 'VW_Display', 'LSTD'] = var1
    var1 = raw_month[(~raw_month['Campagne'].str.contains('MEETDEAL')) & (raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('SOC')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'VW_Paid', 'Direct'] = 0
    df_month.loc[df_month['Focus'] == 'VW_Paid', 'LSTD'] = var1
    var1 = raw_month[(raw_month['Campagne'].str.contains('VW.FR')) & (raw_month['Source'].str.contains('Sans source')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'VW_Organic', 'Direct'] = var1
    df_month.loc[df_month['Focus'] == 'VW_Organic', 'LSTD'] = 0
    var1 = raw_month[(raw_month['Campagne'].str.contains('VP_RDV_DISTRIBUTEURS_SITES')) & (raw_month['Statut'] == 1)].shape[0]
    df_month.loc[df_month['Focus'] == 'Dealers', 'Direct'] = var1
    df_month.loc[df_month['Focus'] == 'Dealers', 'LSTD'] = 0
    df_month.loc[df_month['Focus'] == 'TOTAL MONTH', 'Direct'] = df_month.head(8)['Direct'].sum()
    df_month.loc[df_month['Focus'] == 'TOTAL MONTH', 'LSTD'] = df_month.head(8)['LSTD'].sum()
    df_month['Total'] = df_month['Direct'] + df_month['LSTD']
    df_month[['Direct', 'LSTD', 'Total']] = df_month[['Direct', 'LSTD', 'Total']].astype(int).round()
    df_month['Month'] = current_date
    #df_full = df_full.append(df_month).copy()
    return df_month

# RENDU FINAL (COMPLET)

In [199]:
# rendu du tableau avec l'intégralité des lignes de chaque mois entre aout 19 et aout 21 voir plus
df_final = histo_leads('2019-8')
df_final = df_final.append(histo_leads('2019-9')).copy()
df_final = df_final.append(histo_leads('2019-10')).copy()
df_final = df_final.append(histo_leads('2019-11')).copy()
df_final = df_final.append(histo_leads('2019-12')).copy()
df_final = df_final.append(histo_leads('2020-1')).copy()
df_final = df_final.append(histo_leads('2020-2')).copy()
df_final = df_final.append(histo_leads('2020-3')).copy()
df_final = df_final.append(histo_leads('2020-4')).copy()
df_final = df_final.append(histo_leads('2020-5')).copy()
df_final = df_final.append(histo_leads('2020-6')).copy()
df_final = df_final.append(histo_leads('2020-7')).copy()
df_final = df_final.append(histo_leads('2020-8')).copy()
df_final = df_final.append(histo_leads('2020-9')).copy()
df_final = df_final.append(histo_leads('2020-10')).copy()
df_final = df_final.append(histo_leads('2020-11')).copy()
df_final = df_final.append(histo_leads('2020-12')).copy()
df_final = df_final.append(histo_leads('2021-1')).copy()
df_final = df_final.append(histo_leads('2021-2')).copy()
df_final = df_final.append(histo_leads('2021-3')).copy()
df_final = df_final.append(histo_leads('2021-4')).copy()
df_final = df_final.append(histo_leads('2021-5')).copy()
df_final = df_final.append(histo_leads('2021-6')).copy()
df_final = df_final.append(histo_leads('2021-7')).copy()
df_final = df_final.append(histo_leads('2021-8')).copy()
df_final = df_final.append(histo_leads('2021-9')).copy()
df_final = df_final.append(histo_leads('2021-10')).copy()
df_final = df_final.append(histo_leads('2021-11')).copy()
df_final = df_final.append(histo_leads('2021-12')).copy()
df_final = df_final.append(histo_leads('2022-1')).copy()
df_final = df_final.append(histo_leads('2022-2')).copy()
df_final = df_final.append(histo_leads('2022-3')).copy()
df_final

,Month,Focus,Direct,LSTD,Total
0,2019-8,Affiliation,0,3031,3031
1,2019-8,Lead_Ad,0,371,371
2,2019-8,Operations,0,0,0
4,2019-8,VW_SEA,3360,0,3360
5,2019-8,VW_Display,0,208,208
...,...,...,...,...,...
5,2022-3,VW_Display,0,121,121
6,2022-3,VW_Paid,0,99,99
7,2022-3,VW_Organic,4305,0,4305
8,2022-3,Dealers,199,0,199


In [200]:
# rendu avec seulement les totaux que l'on met dans l'historique donnée
df_final[df_final['Focus'] == 'TOTAL MONTH']

,Month,Focus,Direct,LSTD,Total
9,2019-8,TOTAL MONTH,10402,3610,14012
9,2019-9,TOTAL MONTH,12184,6289,18473
9,2019-10,TOTAL MONTH,2016,5624,7640
9,2019-11,TOTAL MONTH,6198,4264,10462
9,2019-12,TOTAL MONTH,2902,5373,8275
9,2020-1,TOTAL MONTH,24123,13524,37647
9,2020-2,TOTAL MONTH,1903,4687,6590
9,2020-3,TOTAL MONTH,1750,2337,4087
9,2020-4,TOTAL MONTH,1111,0,1111
9,2020-5,TOTAL MONTH,9621,3629,13250


In [202]:
# export des dataframe en excel pour remplir le excel du réseau
df_final.to_excel('../output/leads_data_cleaned.xlsx')
df_final[df_final['Focus'] == 'TOTAL MONTH'].to_excel('../output/leads_data_cleaned_detailed.xlsx')